# Problem 14.14 Refrigeration Cycle Calculation Using the Peng-Robinson EOS

A refrigerator uses the refrigerant R-12, dichlorodifluoromethane. The steps and conditions of the cycle are as follows:
    
* Isobaric condensation to saturation temperature 30°C
* Adiabatic  let-down to P2 = 20 degrees subcooling
* Isobaric evaporation to saturation temperature of 20 °C
* Isentropic compression to P4 = 30 °C

Use the Peng-Robinson EOS.

## Solution

This is quite straightforward, with the only complication coming from the degrees of subcooling.

In [1]:
# Set the conditions and imports
from scipy.constants import bar
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CEOSGas, FlashPureVLS
fluid = 'dichlorodifluoromethane'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

zs = [1]

eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)
liquid = CEOSLiquid(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases,
                    eos_kwargs=eos_kwargs)
gas = CEOSGas(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, 
              eos_kwargs=eos_kwargs)
flasher = FlashPureVLS(constants, correlations, liquids=[liquid], gas=gas, solids=[])

T1 = 273.15+30
state_1 = flasher.flash(VF=0, T=T1)
saturation_state_1 = flasher.flash(T=-20+273.15, VF=1)
# Wording is unclear for state 2 but thermodynamically his is what makes sense
state_2 = flasher.flash(H=state_1.H(), P=saturation_state_1.P)
# Check the flash lowers the pressure
assert state_2.P < state_1.P
state_3 = flasher.flash(P=state_2.P, VF=1)
saturation_state_2 = flasher.flash(T=30+273.15, VF=1)
state_4 = flasher.flash(P=saturation_state_2.P, S=state_3.S())
states = [state_1, state_2, state_3, state_4]

condensation_duty = (state_1.H() - state_4.H())
heating_duty = state_3.H() - state_2.H()
compressing_duty = state_4.H() - state_3.H()
condensation_duty, heating_duty, compressing_duty

(-17303.632306857875, 13889.615500762327, 3414.0168060955466)

In [2]:
# Check the cycle convergence
cycle_error = sum([condensation_duty, heating_duty, compressing_duty])
cycle_error

-9.094947017729282e-13

In [3]:
for state in states:
    print(f'T={state.T:.2f} K, P={state.P:.2f} Pa, VF={state.VF:.2f}, S={state.S():.2f} J/(mol*K), H={state.H():.2f} J/(mol)') 

T=303.15 K, P=742091.64 Pa, VF=0.00, S=-72.41 J/(mol*K), H=-17294.43 J/(mol)
T=253.15 K, P=150606.23 Pa, VF=0.29, S=-70.23 J/(mol*K), H=-17294.43 J/(mol)
T=253.15 K, P=150606.23 Pa, VF=1.00, S=-15.37 J/(mol*K), H=-3404.81 J/(mol)
T=311.88 K, P=742091.64 Pa, VF=1.00, S=-15.37 J/(mol*K), H=9.20 J/(mol)
